In [1]:
# ! pip install cardea
# ! pip install 'urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1'

In [2]:
# imports 

import pandas as pd
import numpy as np

#Cardea
from cardea import Cardea

#Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier

#Evaluation
from sklearn.metrics import accuracy_score

In [3]:
# optional
import warnings
warnings.filterwarnings("ignore")

In [4]:
cd = Cardea()

In [5]:
# ! curl -O https://dai-cardea.s3.amazonaws.com/kaggle.zip && unzip kaggle.zip

In [6]:
cd.load_entityset(data='kaggle')

# to view the loaded entityset
cd.es

Entityset: fhir
  Entities:
    Reference [Rows: 6100, Columns: 1]
    Address [Rows: 81, Columns: 2]
    Identifier [Rows: 227151, Columns: 1]
    Patient [Rows: 6100, Columns: 4]
    Appointment_Participant [Rows: 6100, Columns: 2]
    Observation [Rows: 110527, Columns: 3]
    CodeableConcept [Rows: 4, Columns: 2]
    Appointment [Rows: 110527, Columns: 5]
    Coding [Rows: 3, Columns: 2]
  Relationships:
    Patient.address -> Address.object_id
    Appointment_Participant.actor -> Reference.identifier
    Observation.code -> CodeableConcept.object_id
    Observation.subject -> Reference.identifier
    CodeableConcept.coding -> Coding.object_id
    Appointment.participant -> Appointment_Participant.object_id

In [7]:
cd.list_problems()

{'DiagnosisPrediction',
 'LengthOfStay',
 'MissedAppointment',
 'MortalityPrediction',
 'ProlongedLengthOfStay',
 'Readmission'}

In [8]:
# select problem
label_times = cd.select_problem('MissedAppointment')

In [9]:
# feature engineering
feature_matrix = cd.generate_features(label_times[:1000]) # takes a while for the full dataset
feature_matrix.head(5)

Built 13 features
Elapsed: 00:32 | Progress: 100%|██████████


,participant = 2680425062,participant = 4275143764,participant = 2615334244,participant = 2410824900,participant = 2406221984,participant = 1868414665,participant = 1692482157,participant = 1125465544,participant = 846537388,participant = 4121228070,participant is unknown,DAY(created) = 29,DAY(created) = 28,DAY(created) = 27,DAY(created) = 18,DAY(created) = 15,DAY(created) = 26,DAY(created) = 25,DAY(created) = 5,DAY(created) = 1,DAY(created) = 8,DAY(created) is unknown,DAY(start) = 29,DAY(start) is unknown,IS_WEEKEND(created),IS_WEEKEND(start),MONTH(created) = 4,MONTH(created) = 3,MONTH(created) = 2,MONTH(created) = 1,MONTH(created) is unknown,MONTH(start) = 4,MONTH(start) is unknown,WEEKDAY(created) = 4,WEEKDAY(created) = 2,WEEKDAY(created) = 1,WEEKDAY(created) = 3,WEEKDAY(created) = 0,WEEKDAY(created) is unknown,WEEKDAY(start) = 4,WEEKDAY(start) is unknown,YEAR(created) = 2016,YEAR(created) is unknown,YEAR(start) = 2016,YEAR(start) is unknown,Appointment_Participant.actor = 74200000000000,Appointment_Participant.actor = 713000000000000,Appointment_Participant.actor = 41400000000000,Appointment_Participant.actor = 28200000000000,Appointment_Participant.actor = 7270000000000,Appointment_Participant.actor = 7230000000000,Appointment_Participant.actor = 4920000000000,Appointment_Participant.actor = 3880000000000,Appointment_Participant.actor = 2760000000000,Appointment_Participant.actor = 735000000000000,Appointment_Participant.actor is unknown,Appointment_Participant.COUNT(Appointment),label
0,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,39,noshow
1,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,27,noshow
2,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,55,noshow
3,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,39,noshow
4,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,28,noshow


In [10]:
# shuffle the dataframe
feature_matrix = feature_matrix.sample(frac=1)

# pop the target labels
y = list(feature_matrix.pop('label'))
X = feature_matrix.values

X_train, X_test, y_train, y_test = cd.train_test_split(X, y, test_size=0.2, shuffle=True)

In [11]:
# modeling
cd.select_pipeline('Random Forest')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [12]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.77, 'Confusion Matrix': array([[  3,  30],
        [ 16, 151]]), 'F1 Macro': 0.4916003536693192, 'Precision': 0.49607444024425706, 'Recall': 0.497550353837779}

In [13]:
# modeling1
cd.select_pipeline('Logistic Regression')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [14]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.83, 'Confusion Matrix': array([[  0,  34],
        [  0, 166]]), 'F1 Macro': 0.45355191256830596, 'Precision': 0.415, 'Recall': 0.5}

In [15]:
# modeling2
cd.select_pipeline('Gaussian Naive Bayes')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [16]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.175, 'Confusion Matrix': array([[ 28,   0],
        [165,   7]]), 'F1 Macro': 0.16580297783058218, 'Precision': 0.572538860103627, 'Recall': 0.5203488372093024}

In [17]:
# modeling3
cd.select_pipeline('Gradient Boosting')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [18]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.78, 'Confusion Matrix': array([[  0,  44],
        [  0, 156]]), 'F1 Macro': 0.43820224719101125, 'Precision': 0.39, 'Recall': 0.5}

In [19]:
# modeling4
cd.select_pipeline('K-Nearest Neightbors')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [20]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.775, 'Confusion Matrix': array([[  4,  34],
        [ 11, 151]]), 'F1 Macro': 0.5106301995541297, 'Precision': 0.5414414414414415, 'Recall': 0.5186809616634178}

In [21]:
# modeling5
cd.select_pipeline('Multinomial Naive Bayes')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [22]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.815, 'Confusion Matrix': array([[  4,  32],
        [  5, 159]]), 'F1 Macro': 0.5367762128325508, 'Precision': 0.6384525887143688, 'Recall': 0.5403116531165312}

In [23]:
# modeling6
cd.select_pipeline('Stochastic Gradient Descent')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [24]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.835, 'Confusion Matrix': array([[  0,  33],
        [  0, 167]]), 'F1 Macro': 0.4550408719346049, 'Precision': 0.4175, 'Recall': 0.5}

In [25]:
# modeling7
cd.select_pipeline('XGB')
cd.fit(X_train, y_train)
y_pred = cd.predict(X_test)

In [26]:
cd.evaluate(X, y, test_size=0.2, shuffle=True)

{'Accuracy': 0.83, 'Confusion Matrix': array([[  0,  34],
        [  0, 166]]), 'F1 Macro': 0.45355191256830596, 'Precision': 0.415, 'Recall': 0.5}

In [27]:
#Import GridSearh & RandomSearch
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [28]:
# PARAMETER

In [29]:
param_rf = [{'criterion': ("gini","entropy"), 'n_estimators': (10, 100, 1000)}] #Parameter Random Forest
param_lr = [{'C': np.logspace(-3,3,5,7), "penalty": ["l1","l2"], "fit_intercept": ['True', 'False']}] #Parameter Logistic Regression
param_gnb = [{'var_smoothing': [1e-9]}] #Parameter Gaussian Naive Bayes
param_gb = [{'learning_rate': [0.001, 0.01, 0.1, 0.3], 'n_estimators': [10, 100, 1000]}] #Parameter Gradient Boosting
param_knn = [{'n_neighbors': [5], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}] #Parameter k-nearest neighbors
param_mnb = [{'alpha': [1.0, 0.01, 0.001], "fit_prior": ['True', 'False']}] #Parameter Multinomial Naive Bayes
param_sgd = [{'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'], 'penalty': ['l2', 'l1', 'elasticnet']}] #Parameter Stochastic Gradient Descent
param_xgb = [{'booster': ['gbtree', 'gblinear']}]

In [30]:
#GridSearch Random Forest

In [31]:
gridsearch = GridSearchCV(RandomForestClassifier(), param_rf, scoring='accuracy')
gridsearch.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [32]:
#GridSearch Logistic Regression

In [33]:
gridsearch1 = GridSearchCV(LogisticRegression(), param_lr, scoring='accuracy')
gridsearch1.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': array([1.00000000e-03, 3.16227766e-02, 1.00000000e+00, 3.16227766e+01,
       1.00000000e+03]),
                          'fit_intercept': ['True', 'False'],
                          'penalty': ['l1', 'l2']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring

In [34]:
#GridSearch Gaussian Naive Bayes

In [35]:
gridsearch2 = GridSearchCV(GaussianNB(), param_gnb, scoring='accuracy')
gridsearch2.fit(X_train, y_train) # LR

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=GaussianNB(priors=None, var_smoothing=1e-09), iid='warn',
             n_jobs=None, param_grid=[{'var_smoothing': [1e-09]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [36]:
#GridSearch Gradient Boosting

In [37]:
gridsearch3 = GridSearchCV(GradientBoostingClassifier(), param_gb, scoring='accuracy')
gridsearch3.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  pre

In [38]:
#GridSearch K-Nearest Neightbors

In [39]:
gridsearch4 = GridSearchCV(KNeighborsClassifier(), param_knn, scoring='accuracy')
gridsearch4.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'n_neighbors': [5],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [40]:
# GridSearch Multinomial Naive Bayes

In [41]:
gridsearch5 = GridSearchCV(MultinomialNB(), param_mnb, scoring='accuracy')
gridsearch5.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='warn', n_jobs=None,
             param_grid=[{'alpha': [1.0, 0.01, 0.001],
                          'fit_prior': ['True', 'False']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [42]:
# GridSearch Stochastic Gradient Descent

In [43]:
gridsearch6 = GridSearchCV(SGDClassifier(), param_sgd, scoring='accuracy')
gridsearch6.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5,
                                     random_state=None, shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'loss': ['hinge', 'log', 'modified_huber',
                                   'squared_hinge', 'perceptron', 'huber',
                                   'epsilon_insens

In [44]:
# GridSearch XGB

In [45]:
gridsearch7 = GridSearchCV(XGBClassifier(), param_xgb, scoring='accuracy')
gridsearch7.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid=[{'booster': ['gbtree', 'gblinear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='a

In [46]:
#Hasil GridSearch

In [47]:
print (gridsearch.best_params_) # RF
print(gridsearch1.best_params_) # LR
print(gridsearch2.best_params_) # GNB
print(gridsearch3.best_params_) # GB
print(gridsearch4.best_params_) # KNN
print(gridsearch5.best_params_) # MNB
print(gridsearch6.best_params_) # SGD
print(gridsearch7.best_params_) # XGB

{'criterion': 'gini', 'n_estimators': 100}
{'C': 0.001, 'fit_intercept': 'True', 'penalty': 'l1'}
{'var_smoothing': 1e-09}
{'learning_rate': 0.001, 'n_estimators': 10}
{'algorithm': 'brute', 'n_neighbors': 5, 'weights': 'uniform'}
{'alpha': 1.0, 'fit_prior': 'True'}
{'loss': 'epsilon_insensitive', 'penalty': 'l2'}
{'booster': 'gblinear'}


In [48]:
#RandomSearch Random Forest

In [49]:
randomsearch = RandomizedSearchCV(RandomForestClassifier(), param_rf, scoring='accuracy')
randomsearch

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

In [50]:
#RandomSearch Logistic Regression

In [51]:
randomsearch1 = RandomizedSearchCV(LogisticRegression(), param_lr, scoring='accuracy')
randomsearch1

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='warn', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='warn', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions=[{'C': array([1.00000000e-03, 3.16227766e-02, 1.00000000e+00, 3.16227766e+01,
       1.00000000e+03]),
                                         'fit_intercept': ['True', 'False'],
                                         'penalt

In [52]:
#RandomSearch Gaussian Naive Bayes

In [53]:
randomsearch2 = RandomizedSearchCV(GaussianNB(), param_gnb, scoring='accuracy')
randomsearch2

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions=[{'var_smoothing': [1e-09]}],
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=0)

In [54]:
#GridSearch Gradient Boosting

In [55]:
gridsearch3 = GridSearchCV(GradientBoostingClassifier(), param_gb, scoring='accuracy')
gridsearch3

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  pre

In [56]:
#GridSearch K-Nearest Neightbors

In [57]:
gridsearch4 = GridSearchCV(KNeighborsClassifier(), param_knn, scoring='accuracy')
gridsearch4

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'n_neighbors': [5],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [58]:
# GridSearch Multinomial Naive Bayes

In [59]:
gridsearch5 = GridSearchCV(MultinomialNB(), param_mnb, scoring='accuracy')
gridsearch5

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='warn', n_jobs=None,
             param_grid=[{'alpha': [1.0, 0.01, 0.001],
                          'fit_prior': ['True', 'False']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [60]:
# GridSearch Stochastic Gradient Descent

In [61]:
gridsearch6 = GridSearchCV(SGDClassifier(), param_sgd, scoring='accuracy')
gridsearch6

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5,
                                     random_state=None, shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'loss': ['hinge', 'log', 'modified_huber',
                                   'squared_hinge', 'perceptron', 'huber',
                                   'epsilon_insens

In [62]:
# GridSearch XGB

In [63]:
gridsearch7 = GridSearchCV(XGBClassifier(), param_xgb, scoring='accuracy')
gridsearch7

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid=[{'booster': ['gbtree', 'gblinear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='a